# Shopping Mall Crawling
* Made by Cheonsol Lee
* Updated (20.08.11)

## Import
- selenium
- chrome drvier를 설치해야함

In [93]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### 함수 정의부

In [160]:
def crawling(file_path, site_name, keyword, chromedriver_path):
    file_name = file_path + site_name + '_' + keyword + '.csv'
    
    # 쇼핑몰 종류에 따라 다르게 크롤링
    # Auction은 정적크롤링(BeautifulSoup)
    if(site_name == "Auction"):
        print("쇼핑몰 : Auction!")
        res = requests.get('http://browse.auction.co.kr/search?keyword=' + keyword)
        soup = BeautifulSoup(res.content, 'html.parser')
        items = soup.select("div.section--itemcard")
        
        title_list = [item.select("div.area--itemcard_title span.text--title")[0].get_text() for item in items]
        price_list = [item.select("strong.text--price_seller")[0].get_text() for item in items]
        seller_list = [item.select("a.link--shop span")[1].get_text() for item in items]
        
        df = pd.DataFrame({'title': title_list,'price': price_list, 'seller': seller_list})
        df.index.name = 'index'
    
    # Danawa는 동적크롤링(selenium, BeautifulSoup)
    if(site_name == "Danawa"):
        print("쇼핑몰 : Danawa!")
        driver = webdriver.Chrome(chromedriver_path)
        driver.get("http://search.danawa.com/dsearch.php?query=" + keyword)
        
        # 명시적 대기 : 특정 태그가 로드될 때까지 대기
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'prod_name')))
        
        html = driver.page_source

        soup = BeautifulSoup(html, 'html.parser')
        items = soup.select("div.main_prodlist li.prod_item")

        title_list = [item.select('p.prod_name a')[0].text for item in items]
        price_list = [item.select('p.price_sect a strong')[0].text for item in items]
        volume_list = [item.select('p.prod_name a')[0].text.split(" ")[-1] for item in items]
        
        df = pd.DataFrame({'title': title_list,'price': price_list, 'volume': volume_list})
        df.index.name = 'index'

        
    if(site_name == "Amazone"):
        print("쇼핑몰 : Amazone!")
        driver = webdriver.Chrome(chromedriver_path)
        driver.get("https://www.amazon.com/s?k=" + keyword)

        # 명시적 대기 : 특정 태그가 로드될 때까지 대기
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'nav-input')))

        html = driver.page_source

        soup = BeautifulSoup(html, 'html.parser')
        items = soup.select("div.sg-col-inner div.sg-col-inner")
        items = items[ : -1] # 마지막은 상품명이 아님.
        
        title_list = [item.select('h2 span')[0].text for item in items]
        price_list = []
        for item in items:
            try:
                item.select("span.a-price span.a-offscreen")[0].get_text()
                price = item.select("span.a-price span.a-offscreen")[0].get_text()
                price_list.append(price)
            except:
                price_list.append(0)
                
        df = pd.DataFrame({'title': title_list,'price': price_list})
        df.index.name = 'index'
    
    
    print("-----------------------아래는 테이블 구조입니다.-----------------------")
    print(df.head())
    print("--------------------------------------------------------------------")
    
    df.to_csv(file_name, mode = 'w', encoding ='utf-8-sig', index = True, header = True)
    print("저장되었습니다.파일을 확인해주세요.")

### 사용자정의 입력부
- file_path : 저장위치 디렉토리
- site_name : 'Aution' or 'Danawa'입력(2개만 가능)
- keyword   : 'hemp', 'bean' 등 자유롭게 입력가능
- chromedriver_path : 크롬드라이버가 설치된 절대경로

In [161]:
# 파일명 형태 : Auction_hemp.csv
file_path = 'C:/Users/KSE/JupyterProjects/knowledge_structure/'
site_name = 'Amazone'
keyword = 'hemp'

# 크롬드라이버 절대경로
chromedriver_path = 'C:/Users/KSE/Desktop/crawling/chromedriver.exe'

### 실행부

In [162]:
crawling(file_path, site_name, keyword, chromedriver_path)

쇼핑몰 : Amazone!
-----------------------아래는 테이블 구조입니다.-----------------------
                                                   title   price
index                                                           
0      Hempz Sweet Pineapple & Honey Melon Moisturizi...  $19.51
1      Manitoba Harvest Organic Hemp Hearts Shelled H...  $13.94
2      Premium Hemp Gummies - Natural Hemp - Made in ...  $25.49
3      Manitoba Harvest Hemp Hearts Shelled Hemp Seed...       0
4      Hemp Cream - 2,000,000 - Relieve Muscle, Joint...  $22.95
--------------------------------------------------------------------
저장되었습니다.파일을 확인해주세요.
